# BS Optimization

- This script will take a fake BS and perform an optimization to determine highest profit generating balances
- After the first successful run, I will layer on additional complexity in terms of contraints

In [139]:
import pandas as pd
import numpy as np
import scipy.optimize as spo

In [140]:
# Define objective function
# Only used in the version of that uses the 'minimize' function

# def fun(x):
    
#     sva = x * nsi
    
#     return -sva

In [141]:
# Define starting balances
# Not need for linprog implementation

# x0 = np.array(df['start'])

# x0.reshape(-1, 1)

In [142]:
# Read in BS data

df = pd.read_excel('./data/sample_bs.xlsx', index_col=0, nrows=29)

In [143]:
# Read in constraints

constraints = pd.read_excel('./data/sample_bs.xlsx', index_col=0, sheet_name='constraints')

In [144]:
constraints

,CET1,T1,total_capital,TLAC
SRWA,0.11,0.125,0.145,0.225
ARWA,0.11,0.125,0.145,0.225


In [145]:
df.head()

,Product,start,grow,shrink,spread,A_L,a_rwa,s_rwa,CET1_resource,T1_resource,total_capital_resource,TLAC_resource
Index,,,,,,,,,,,,
1,prime_auto,15000,8000,-5000,80,1,0.2,1.0,0,0,0,0
2,subprime_auto,4000,2000,-1000,150,1,0.6,1.0,0,0,0,0
3,mtg_30_fixed,50000,30000,-15000,70,1,0.3,0.5,0,0,0,0
4,mtg_15_fixed,15000,8000,-3000,72,1,0.3,0.5,0,0,0,0
5,mtg_7_fixed,5000,2500,-2000,70,1,0.3,0.5,0,0,0,0


In [146]:
# Define objective function coefficients
# We are 'minimizing' the spread so will make this negative
# Dividing by 10,000 to convert bps to spread (will not affect optimization)

c = -np.array(df['spread'] / 10000)

## Inequality Constraint

- The inequality constraint for our optimization is the the SRWA % * 11% * the balance - equity needs to be greater than or equal to 0.
- Since it is greater than, for purposes of Scipy notation, we need to flip the sign

### Advanced RWA

In [147]:
# Extract Advanced RWA inequality coefficients

aRWA_mins = list(constraints.loc['ARWA', :])

aRWA_cet1 = list(df['a_rwa'] * aRWA_mins[0] - df['CET1_resource'])
aRWA_t1 = list(df['a_rwa'] * aRWA_mins[1] - df['T1_resource'])
aRWA_tc = list(df['a_rwa'] * aRWA_mins[2] - df['total_capital_resource'])
aRWA_tlac = list(df['a_rwa'] * aRWA_mins[3] - df['TLAC_resource'])

# Combine into one list
aRWA_constraints = [aRWA_cet1, aRWA_t1, aRWA_tc, aRWA_tlac]

### Standardized RWA

In [148]:
# Extract RWA inequality coefficients

sRWA_mins = list(constraints.loc['SRWA', :])

sRWA_cet1 = list(df['s_rwa'] * sRWA_mins[0] - df['CET1_resource'])
sRWA_t1 = list(df['s_rwa'] * sRWA_mins[1] - df['T1_resource'])
sRWA_tc = list(df['s_rwa'] * sRWA_mins[2] - df['total_capital_resource'])
sRWA_tlac = list(df['s_rwa'] * sRWA_mins[3] - df['TLAC_resource'])

# Combine into one list

sRWA_constraints = [sRWA_cet1, sRWA_t1, sRWA_tc, sRWA_tlac]

In [149]:
sRWA_tlac

[0.22499999999999998,
 0.22499999999999998,
 0.11249999999999999,
 0.11249999999999999,
 0.11249999999999999,
 0.11249999999999999,
 0.11249999999999999,
 0.22499999999999998,
 0.22499999999999998,
 0.26999999999999996,
 0.22499999999999998,
 0.25875,
 0.22499999999999998,
 0.12375,
 0.13499999999999998,
 0.11249999999999999,
 0.15749999999999997,
 0.01125,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0]

In [150]:
# Combine Advanced RWA and Standardized RWA constraints into a single list

A_ineq = aRWA_constraints + sRWA_constraints

# Define the other side of the inequality equation (we want to be at least at the minimum, so this is 0)
# There are 8 constraints so this gets multiplied accordingly

b_ineq = [0] * 8

In [151]:
A_ineq

[[0.022000000000000002,
  0.066,
  0.033,
  0.033,
  0.033,
  0.033,
  0.033,
  0.12100000000000001,
  0.1045,
  0.0825,
  0.077,
  0.07150000000000001,
  0.066,
  0.044000000000000004,
  0.055,
  0.0385,
  0.055,
  0.0055000000000000005,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -1.0,
  0.0,
  0.0,
  0.0],
 [0.025,
  0.075,
  0.0375,
  0.0375,
  0.0375,
  0.0375,
  0.0375,
  0.1375,
  0.11875,
  0.09375,
  0.0875,
  0.08125,
  0.075,
  0.05,
  0.0625,
  0.04375,
  0.0625,
  0.00625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -1.0,
  -1.0,
  0.0,
  0.0],
 [0.028999999999999998,
  0.087,
  0.0435,
  0.0435,
  0.0435,
  0.0435,
  0.0435,
  0.1595,
  0.13774999999999998,
  0.10874999999999999,
  0.10149999999999999,
  0.09425,
  0.087,
  0.057999999999999996,
  0.0725,
  0.050749999999999997,
  0.0725,
  0.0072499999999999995,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -1.0,
  -1.0,
  -1.0,
  0.0],
 [0.045,
  0.13499999999999998,
  0.06749999999999999,
  0.0674999

## Equality Constraint

- This constraint says that Assets - Liabilities must equal to 0.
- I've pre-programmed this to an extent by including a assets (+100%) and liability (-100%) weight in the inputs.

In [152]:
# Create Asset and Liability equality constraint.

A_eq = [list(df['A_L'])]

In [153]:
# Equality constraint vector defined below. This is saying that assets must equal liabilities on the B/S

a_l_constraint = 0

b_eq = [a_l_constraint]

In [154]:
# Create upper and lower bounds. This is the grow and shrink amount for each item

df['u_bound'] = df['start'] + df['grow']
df['l_bound'] = df['start'] + df['shrink']

# Create tuple of lower / upper bounds
df['bounds'] = df.apply(lambda row: tuple((row['l_bound'], row['u_bound'])), axis=1)

In [155]:
# Define bounds in format needed for linprog

bounds = list(df['bounds'])

## Optimize B/S

In [164]:
res = spo.linprog(c, A_ub = A_ineq, b_ub = b_ineq, A_eq = A_eq, b_eq = b_eq, bounds = bounds, 
                  method='revised simplex')

## Results

In [165]:
# This shows that a solution was found

res.success

True

In [166]:
# Show the output of the objective function

res.fun

-14142.724032258067

In [167]:
# Compare to starting profitability

start_profitability = -sum(df['start'] * (df['spread'] / 10000))

(res.fun - start_profitability) / start_profitability

# Profitability increased 50%

0.36289139753860145

In [171]:
# Show the slack against each of the constraints.
# As expect, Standardized RWA (last four values) is significantly more binding than Advanced

res.slack

array([ 1.63452992e+04,  1.85742036e+04,  2.15460762e+04,  3.34335665e+04,
       -3.63797881e-12, -7.27595761e-12,  0.00000000e+00,  0.00000000e+00])

In [172]:
# Show ending balances

res.x

array([ 10000.        ,   6000.        ,  80000.        ,  23000.        ,
         7500.        ,  30000.        ,  13500.        ,  80000.        ,
        17500.        ,  15000.        ,  51562.09677419,  28000.        ,
        85000.        , 105000.        ,  75000.        ,  95000.        ,
        34000.        ,   1500.        , 235000.        ,  85000.        ,
       130000.        ,  17000.        , 125500.        ,  38000.        ,
         3000.        ,  60652.58064516,   8270.80645161,  11027.74193548,
        44110.96774194])

In [173]:
# Append ending balance to our data and compare growth vs. shrink

df['optimal_balance'] = res.x

df['balance_change'] = df['optimal_balance'] - df['start']

In [174]:
balance_results = df[['Product', 'balance_change']]

In [175]:
# Some products are growing, some are shrinking, and business non-op deposits are our marginal product

balance_results

,Product,balance_change
Index,,
1,prime_auto,-5000.000000
2,subprime_auto,2000.000000
3,mtg_30_fixed,30000.000000
4,mtg_15_fixed,8000.000000
5,mtg_7_fixed,2500.000000
6,mtg_15_arm,10000.000000
7,mtg_7_arm,3500.000000
8,consumer_card,20000.000000
9,business_card,2500.000000


In [176]:
df.head()

,Product,start,grow,shrink,spread,A_L,a_rwa,s_rwa,CET1_resource,T1_resource,total_capital_resource,TLAC_resource,u_bound,l_bound,bounds,optimal_balance,balance_change
Index,,,,,,,,,,,,,,,,,
1,prime_auto,15000,8000,-5000,80,1,0.2,1.0,0,0,0,0,23000,10000,"(10000, 23000)",10000.0,-5000.0
2,subprime_auto,4000,2000,-1000,150,1,0.6,1.0,0,0,0,0,6000,3000,"(3000, 6000)",6000.0,2000.0
3,mtg_30_fixed,50000,30000,-15000,70,1,0.3,0.5,0,0,0,0,80000,35000,"(35000, 80000)",80000.0,30000.0
4,mtg_15_fixed,15000,8000,-3000,72,1,0.3,0.5,0,0,0,0,23000,12000,"(12000, 23000)",23000.0,8000.0
5,mtg_7_fixed,5000,2500,-2000,70,1,0.3,0.5,0,0,0,0,7500,3000,"(3000, 7500)",7500.0,2500.0


In [177]:
# Export updated results to Excel

df.to_excel('./results/results.xlsx')